In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:09<01:18,  2.30it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:42<00:43,  2.25it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [01:06<00:49,  1.39it/s]

...

 79%|███████▉  | 158/200 [01:26<00:14,  2.95it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:51<00:00,  1.79it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 11865.25it/s]


In [7]:
importlib.reload(torch_classes)
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [8]:
model = torch_classes.GRUNetV2(12,64).to(device='cuda:0')

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(12,config['hidden_size']).to('cuda:0')
        print(model)
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [10]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [11]:
config_static = {'learning_rate':0.00001, 'hidden_size':128, 'num_layers':1}

In [12]:
trading_data.reset_hidden(128,1)
trading_data.detach_hidden()

In [13]:
model

GRUNetV2(
  (gru): GRU(12, 64, dropout=0.3)
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(12800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): LeakyReLU(negative_slope=0.01)
  (fc0): Linear(in_features=12800, out_features=200, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)

In [14]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]]
X = trading_data.packed_val_x[0] 
Y = trading_data.packed_val_y[0].data

In [15]:
# hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

In [16]:
# hidden_in 

In [17]:
# output,hidden = model(X,hidden_in,p1=True)

In [18]:
model = torch_classes.GRUNetV2(12,64)
model_loc = f"C:/Users/Nick/Documents/GitHub/OptiverKaggle/models/tough-totem-105/tough-totem-105_160.pt"
model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
print(model_data['model_state_dict'].keys())

print(model_data['model_state_dict'].keys())
del_keys = ['fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias']
[model_data['model_state_dict'].pop(k) for k in del_keys]
model.load_state_dict(model_data['model_state_dict'], strict=False)
model = model.to(device='cuda:0')

odict_keys(['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])
odict_keys(['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'fc0.weight', 'fc0.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


In [19]:
model = model_pipeline(trading_data, config_static)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


GRUNetV2(
  (gru): GRU(12, 128, dropout=0.3)
  (batch_norm): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm2): BatchNorm1d(25600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu0): LeakyReLU(negative_slope=0.01)
  (fc0): Linear(in_features=25600, out_features=200, bias=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=200, bias=True)
  (rl2): ReLU()
  (drop2): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=640, out_features=200, bias=True)
  (rl3): ReLU()
  (drop3): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=200, out_features=200, bias=True)
)


  0%|          | 0/10000 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_23876\2241259923.py", line 12, in model_pipeline
    training_testing.train_model(trading_df,model,config,optimizer,criterion)
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\training_testing.py", line 37, in train_model
    output,hidden = model(X,hidden_in,p1=True)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\Documents\GitHub\OptiverKaggle\torch_classes.py", line 234, in forward
    x,hidden = self.gru(x,h)
               ^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nick\.conda\envs\python311\Lib

RuntimeError: Expected hidden size (1, 191, 128), got [2, 191, 128]